In [ ]:
import os, re,json, imp, unittest
from glob import glob
import pandas as pd
import warnings

warnings.simplefilter("ignore", ResourceWarning)


print('First Directory at :', os.getcwd()[2:])

#Load Setting File
with open('setting.txt','r') as f:
    setting = json.load(f)
print('Load Setting OK')    

os.chdir(re.sub(os.getcwd().split('\\')[-1]+'$','',os.getcwd()))
print('Change Directory into :', os.getcwd()[2:])

In [ ]:
local_library = True
# local_library = False

if local_library : 
    lazy_GCS = imp.load_source('lazy_gcs', 'py_topping/data_connection/gcp.py').lazy_GCS
else :
    base_cwd = os.getcwd()[:]
    os.chdir(os.getcwd().split('\\')[0] + '\\')
    from py_topping.data_connection.gcp import lazy_GCS
    os.chdir(base_cwd)

# Cloud Storage

In [ ]:
gcs_setting = setting['gcs']

class gcs_test(unittest.TestCase):
    def setUp(self):
        warnings.simplefilter("ignore")
        self.gcs = lazy_gcs(project_id = gcs_setting["project_id"]
                     , bucket_name = gcs_setting["bucket"]
                     , credential = gcs_setting["credentials_path"])
        self.folder = gcs_setting["folder"]
        self.filename = gcs_setting["filename"]

    def test_download(self):
        self.gcs.download(bucket_file = '{}/{}'.format(self.folder,self.filename)
                            , local_file = self.filename)
        result, expect = os.path.isfile(self.filename), True
        self.assertEqual(result, expect)
        os.remove(self.filename)
        
    def test_downloadfolder(self):
        local_folder = self.folder.split('/')[-1]
        self.gcs.download_folder(bucket_folder = self.folder
                                , local_folder = local_folder
                                , create_folder = True)
        downloaded_file = glob('{}/*'.format(local_folder))
        result, expect = len(downloaded_file) > 0, True
        self.assertEqual(result, expect)
        for i in downloaded_file : os.remove(i)
        os.rmdir(local_folder)
        
    def test_upload(self):
        upload_file = 'dummy.txt'
        with open(upload_file, 'w') as f :
            f.write('a')
        self.gcs.upload(bucket_file = '{}/{}'.format(self.folder,upload_file)
           , local_file = upload_file)
        self.gcs.download(bucket_file = '{}/{}'.format(self.folder,upload_file)
                            , local_file = 'new_' + upload_file)
        with open('new_' + upload_file, 'r') as f :
            reread = str(f.read())
        result, expect = reread, 'a'
        self.assertEqual(result, expect)
        os.remove(upload_file) 
        os.remove('new_' + upload_file) 
        
unittest.main(argv = ['first-arg-is-ignored'], exit = False)